In [1]:
import numpy as np
import sys
import torch
import torch.nn as nn
sys.path.append('..')
import MLMD
import matplotlib.pyplot as plt
from MLMD.step import QM_step
import os

首先运行30帧QM

In [2]:
QM_step(30)

 running on   16 total cores
 distrk:  each k-point on   16 cores,    1 groups
 distr:  one band on    4 cores,    4 groups
 using from now: INCAR     
 vasp.5.4.4.18Apr17-6-g9f103f2a35 (build Oct 18 2021 17:02:20) complex          
  
 POSCAR found type information on POSCAR  F  Li
 POSCAR found :  2 types and      62 ions
 scaLAPACK will be used
 LDA part: xc-table for Pade appr. of Perdew
 POSCAR found type information on POSCAR  F  Li
 POSCAR found :  2 types and      62 ions

 ----------------------------------------------------------------------------- 
|                                                                             |
|           W    W    AA    RRRRR   N    N  II  N    N   GGGG   !!!           |
|           W    W   A  A   R    R  NN   N  II  NN   N  G    G  !!!           |
|           W    W  A    A  R    R  N N  N  II  N N  N  G       !!!           |
|           W WW W  AAAAAA  RRRRR   N  N N  II  N  N N  G  GGG   !            |
|           WW  WW  A    A  R   R 

In [3]:
#初始化网络

atom_mass = [19,6.941]

embed_layers = [16,32,32,64]
linear_layers = [64,128,256]

lr = 0.004


In [4]:
#读取XDATCAR

LiF = MLMD.atom_io.loadfile(1,"vasprun/XDATCAR")
traj = MLMD.utilities.get_train_data(LiF)

lattice  = torch.tensor(list(LiF.values())[0].lattice).float()

#制作测试集
train_data = MLMD.utilities.get_train_data(LiF)
v = MLMD.utilities.diff(train_data)
f = MLMD.utilities.get_force_from_v(v,LiF,atom_mass)
#制作原子类型表（对网络重要）
atom_list = MLMD.neighbor_list.get_atom_list(LiF)

#初始化模型

device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLMD.model.DnnFull(atom_list,5,embed_layers,linear_layers,lattice).to(device)

Reading XDATCAR...
F      : 32
Li     : 30
Loading XDATCAR...
done


训练网络

In [5]:
model = MLMD.step.learn_from_QM(model,train_data,f,lr)

0 : Loss: 2.360e+00,time:2.20s
1 : Loss: 2.824e+00,time:2.17s
2 : Loss: 8.850e-01,time:2.18s
3 : Loss: 7.199e-01,time:2.18s
4 : Loss: 4.217e-01,time:2.26s
5 : Loss: 3.249e-01,time:2.17s
6 : Loss: 2.529e-01,time:2.16s
7 : Loss: 2.010e-01,time:2.20s
8 : Loss: 1.865e-01,time:2.20s
9 : Loss: 1.496e-01,time:2.25s
10 : Loss: 1.271e-01,time:2.18s
11 : Loss: 1.036e-01,time:2.15s
12 : Loss: 1.143e-01,time:2.16s
13 : Loss: 1.069e-01,time:2.17s
14 : Loss: 8.764e-02,time:2.26s
15 : Loss: 7.665e-02,time:2.16s
16 : Loss: 8.309e-02,time:2.17s
17 : Loss: 8.366e-02,time:2.19s
18 : Loss: 6.735e-02,time:2.22s
19 : Loss: 7.617e-02,time:2.16s
20 : Loss: 6.367e-02,time:2.18s
21 : Loss: 6.422e-02,time:2.16s
22 : Loss: 5.565e-02,time:2.16s
23 : Loss: 5.864e-02,time:2.24s
24 : Loss: 6.318e-02,time:2.18s
25 : Loss: 6.023e-02,time:2.16s
26 : Loss: 8.431e-02,time:2.16s
27 : Loss: 6.767e-02,time:2.18s
28 : Loss: 6.958e-02,time:2.23s


往后更新10帧

In [6]:
step = 10
predict_coord,predict_vlist = MLMD.step.ML_step_full(model,[traj[-1]],step,LiF,atom_mass)
#写入POSCAR文件
MLMD.utilities.gen_contcar(predict_coord[step],predict_vlist[step-1],LiF,"vasprun/POSCAR")

运行QM来判断结构合理性

In [7]:
QM_step(5)

 running on   16 total cores
 distrk:  each k-point on   16 cores,    1 groups
 distr:  one band on    4 cores,    4 groups
 using from now: INCAR     
 vasp.5.4.4.18Apr17-6-g9f103f2a35 (build Oct 18 2021 17:02:20) complex          
  
 POSCAR found type information on POSCAR  F  Li
 POSCAR found :  2 types and      62 ions
 scaLAPACK will be used
 LDA part: xc-table for Pade appr. of Perdew
 POSCAR found type information on POSCAR  F  Li
 POSCAR found :  2 types and      62 ions

 ----------------------------------------------------------------------------- 
|                                                                             |
|           W    W    AA    RRRRR   N    N  II  N    N   GGGG   !!!           |
|           W    W   A  A   R    R  NN   N  II  NN   N  G    G  !!!           |
|           W    W  A    A  R    R  N N  N  II  N N  N  G       !!!           |
|           W WW W  AAAAAA  RRRRR   N  N N  II  N  N N  G  GGG   !            |
|           WW  WW  A    A  R   R 

In [8]:
LiF = MLMD.atom_io.loadfile(1,"vasprun/XDATCAR")
train_data = MLMD.utilities.get_train_data(LiF)
v = MLMD.utilities.diff(train_data)
f = MLMD.utilities.get_force_from_v(v,LiF,atom_mass)

Reading XDATCAR...
F      : 32
Li     : 30
Loading XDATCAR...
done


In [9]:
test_v_predicted = MLMD.utilities.predict_v_full(model,train_data[0],LiF,atom_mass)
crit = np.linalg.norm(test_v_predicted - v[0])/np.linalg.norm(v[0])
print(crit)

0.3026724613954085


In [11]:
#如果小于容忍值，更新坐标
tol = 0.4
print(len(traj),len(predict_coord),len(train_data))
if crit < tol:
    traj += predict_coord
    traj += train_data

30 11 5


In [12]:
#符合要求的同时进行学习，后续模型的训练采用更小的学习率
model = MLMD.step.learn_from_QM(model,train_data,f,0.001)

0 : Loss: 2.454e-01,time:2.21s
1 : Loss: 1.472e-01,time:2.20s
2 : Loss: 1.306e-01,time:2.18s
3 : Loss: 1.406e-01,time:2.25s


重复上述步骤,直到跑一百步

In [13]:
cirtlist = []
while len(traj)<100:
    
    step = 10
    predict_coord,predict_vlist = MLMD.step.ML_step_full(model,[traj[-1]],step,LiF,atom_mass)
    #写入POSCAR文件
    MLMD.utilities.gen_contcar(predict_coord[step],predict_vlist[step-1],LiF,"vasprun/POSCAR")
    QM_step(5)

    LiF = MLMD.atom_io.loadfile(1,"vasprun/XDATCAR")
    train_data = MLMD.utilities.get_train_data(LiF)
    v = MLMD.utilities.diff(train_data)
    f = MLMD.utilities.get_force_from_v(v,LiF,atom_mass)
    test_v_predicted = MLMD.utilities.predict_v_full(model,train_data[0],LiF,atom_mass)
    crit = np.linalg.norm(test_v_predicted - v[0])/np.linalg.norm(v[0])
    cirtlist.append(crit)
    print(crit)
    #如果小于容忍值，更新坐标（还没写else，先不管）
    tol = 1
    print(len(traj),len(predict_coord),len(train_data))
    if crit < tol:
        traj += predict_coord
        traj += train_data       
        #符合要求的同时进行学习，后续模型的训练采用更小的学习率
        model = MLMD.step.learn_from_QM(model,train_data,f,0.001)
    

 running on   16 total cores
 distrk:  each k-point on   16 cores,    1 groups
 distr:  one band on    4 cores,    4 groups
 using from now: INCAR     
 vasp.5.4.4.18Apr17-6-g9f103f2a35 (build Oct 18 2021 17:02:20) complex          
  
 POSCAR found type information on POSCAR  F  Li
 POSCAR found :  2 types and      62 ions
 scaLAPACK will be used
 LDA part: xc-table for Pade appr. of Perdew
 POSCAR found type information on POSCAR  F  Li
 POSCAR found :  2 types and      62 ions

 ----------------------------------------------------------------------------- 
|                                                                             |
|           W    W    AA    RRRRR   N    N  II  N    N   GGGG   !!!           |
|           W    W   A  A   R    R  NN   N  II  NN   N  G    G  !!!           |
|           W    W  A    A  R    R  N N  N  II  N N  N  G       !!!           |
|           W WW W  AAAAAA  RRRRR   N  N N  II  N  N N  G  GGG   !            |
|           WW  WW  A    A  R   R 

In [14]:
cirtlist

[0.172151570383714,
 0.27191425635266714,
 0.2245872942813463,
 0.22720311927883163]

In [15]:
MLMD.utilities.gen_xdatcar(traj,LiF)